### Ejercicio 3 : (Este ejercicio debe realizarse con archivos csv,json,parquet)
- [x] Leer archivo en DataFrame
- [x] Desplegar DataFrame
- [x] Escribir DataFrame en archivo

In [24]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder \
.appName("Exercise3") \
.getOrCreate()

In [25]:
df2_csv = spark.read.csv("data2.csv", header=True, inferSchema=True)
df_csv = spark.read.csv("data.csv", header=True, inferSchema=True)

print("DataFrame CSV as part:")
df2_csv.show()

print("DataFrame CSV as single file:")
df_csv.show()

DataFrame CSV as part:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|Shirley| 22|     M|
|Jasmine| 50|     M|
|  David| 19|     M|
|   Mark| 25|     M|
+-------+---+------+

DataFrame CSV as single file:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|  David| 19|     M|
|   Mark| 25|     M|
|Shirley| 22|     M|
|Jasmine| 50|     M|
+-------+---+------+



In [26]:
df2_json = spark.read.json("data2.json")
df_json = spark.read.json("data.json", multiLine=True)

print("DataFrame JSON as part:")
df2_json.show()

print("DataFrame JSON as single file:")
df_json.show()

df_json.printSchema()

DataFrame JSON as part:
+---+------+-------+
|Age|Gender|   Name|
+---+------+-------+
| 22|     M|Shirley|
| 50|     M|Jasmine|
| 19|     M|  David|
| 25|     M|   Mark|
+---+------+-------+

DataFrame JSON as single file:
+---+------+-------+
|Age|Gender|   Name|
+---+------+-------+
| 19|     M|  David|
| 25|     M|   Mark|
| 22|     M|Shirley|
| 50|     M|Jasmine|
+---+------+-------+

root
 |-- Age: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Name: string (nullable = true)



In [27]:
df_parquet = spark.read.parquet("data2.parquet")
df2_parquet = spark.read.parquet("data2.parquet")

print("DataFrame Parquet as part:")
df_parquet.show()

print("DataFrame Parquet as single file:")
df2_parquet.show()

DataFrame Parquet as part:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|Jasmine| 50|     M|
|Shirley| 22|     M|
|  David| 19|     M|
|   Mark| 25|     M|
+-------+---+------+

DataFrame Parquet as single file:
+-------+---+------+
|   Name|Age|Gender|
+-------+---+------+
|Jasmine| 50|     M|
|Shirley| 22|     M|
|  David| 19|     M|
|   Mark| 25|     M|
+-------+---+------+



In [29]:
df_csv.write.csv("result_csv", header=True)
df_csv.coalesce(1).write.csv("result2_csv", header=True)

In [30]:
json_list = df_json.toPandas().to_dict(orient="records")

with open("result2.json", "w") as json_file:
    json.dump(json_list, json_file, indent=4)



num_records = df_json.count()
df_repartitioned = df_json.repartition(num_records)
df_repartitioned.write.mode("overwrite").json("partitioned_json_data")

In [31]:
df_parquet.write.parquet("result_parquet")
df_parquet.coalesce(1).write.parquet("result2_parquet")

In [32]:
import os
import shutil

def move_single_file(temp_dir, output_file):
    for file_name in os.listdir(temp_dir):
        if file_name.endswith(".json") or file_name.endswith(".csv") or file_name.endswith(".parquet"):
            shutil.move(os.path.join(temp_dir, file_name), output_file)
            break
    shutil.rmtree(temp_dir)

move_single_file("result2_csv", "result.csv")
move_single_file("result2_parquet", "result.parquet")

In [33]:
spark.stop()